In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, TextBox
from matplotlib.animation import FuncAnimation

In [ ]:
dt = 0.5
cpu_target = 0.60
cooldown_steps = int(5 / dt)
pods_actuales = 2
ultimo_escaleo = -cooldown_steps
tiempo_actual = 0.0
ddos_activado = False
ruido_activado = True
min_pods = 2
max_pods = 10

tiempos, uso_cpu, cantidad_pods, carga_registrada = [], [], [], []

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 9), sharex=True)
plt.subplots_adjust(bottom=0.35)

line_cpu, = ax1.plot([], [], color='orange', label='CPU por pod (%)')
ax1.axhline(cpu_target * 100, color='r', linestyle='--', label='Objetivo CPU')
ax1.set_ylabel('CPU por pod (%)')
ax1.set_ylim(0, 220)
ax1.legend(); ax1.grid(True)
text_cpu = ax1.text(-5, 0, '', fontsize=10, va='center', color='black')

line_pods, = ax2.plot([], [], color='green', label='Pods activos')
ax2.set_ylabel('Pods activos'); ax2.set_ylim(0, 12)
ax2.legend(); ax2.grid(True)
text_pods = ax2.text(-5, 0, '', fontsize=10, va='center', color='black')

line_carga, = ax3.plot([], [], color='blue', label='Carga total')
ax3.set_ylabel('Carga total'); ax3.set_xlabel('Tiempo (min)')
ax3.legend(); ax3.grid(True)

In [ ]:
slider_carga_ax = plt.axes([0.35, 0.27, 0.55, 0.03])
slider_carga = Slider(slider_carga_ax, 'Carga', 0.0, 5.0, valinit=0.5, valstep=0.1)
textbox_carga_ax = plt.axes([0.25, 0.27, 0.08, 0.03])
textbox_carga = TextBox(textbox_carga_ax, '', initial='0.5')

slider_kp_ax = plt.axes([0.35, 0.22, 0.55, 0.03])
slider_kp = Slider(slider_kp_ax, 'Kp', 0.0, 50.0, valinit=10.0, valstep=1)
textbox_kp_ax = plt.axes([0.25, 0.22, 0.08, 0.03])
textbox_kp = TextBox(textbox_kp_ax, '', initial='10')

def update_slider_carga(text):
    try:
        val = float(text)
        val = max(0.0, min(5.0, val))
        slider_carga.set_val(val)
    except ValueError:
        pass

def update_slider_kp(text):
    try:
        val = float(text)
        val = max(0.0, min(50.0, val))
        slider_kp.set_val(val)
    except ValueError:
        pass

textbox_carga.on_submit(update_slider_carga)
textbox_kp.on_submit(update_slider_kp)
slider_carga.on_changed(lambda val: textbox_carga.set_val(f'{val:.1f}'))
slider_kp.on_changed(lambda val: textbox_kp.set_val(f'{val:.0f}'))

In [ ]:
def toggle_ddos(event):
    global ddos_activado
    ddos_activado = not ddos_activado
    btn_ddos.label.set_text('DDoS ON' if ddos_activado else 'Ataque DDoS')

def toggle_ruido(event):
    global ruido_activado
    ruido_activado = not ruido_activado
    btn_ruido.label.set_text('Ruido: ON' if ruido_activado else 'Ruido: OFF')

def reset_simulacion(event):
    global pods_actuales, ultimo_escaleo, tiempo_actual
    global tiempos, uso_cpu, cantidad_pods, carga_registrada
    global ddos_activado, ruido_activado
    pods_actuales = 2
    ultimo_escaleo = -cooldown_steps
    tiempo_actual = 0.0
    ddos_activado = False
    ruido_activado = True
    tiempos.clear(); uso_cpu.clear(); cantidad_pods.clear(); carga_registrada.clear()
    slider_carga.set_val(0.5)
    slider_kp.set_val(10.0)
    btn_ddos.label.set_text('Ataque DDoS')
    btn_ruido.label.set_text('Ruido: ON')
    line_cpu.set_data([], []); line_pods.set_data([], []); line_carga.set_data([], [])
    text_cpu.set_text(''); text_pods.set_text('')
    fig.canvas.draw_idle()

btn_ddos = Button(plt.axes([0.15, 0.1, 0.15, 0.04]), 'Ataque DDoS'); btn_ddos.on_clicked(toggle_ddos)
btn_ruido = Button(plt.axes([0.35, 0.1, 0.20, 0.04]), 'Ruido: ON'); btn_ruido.on_clicked(toggle_ruido)
btn_reset = Button(plt.axes([0.60, 0.1, 0.15, 0.04]), 'Resetear'); btn_reset.on_clicked(reset_simulacion)

In [ ]:
def update(frame):
    global pods_actuales, ultimo_escaleo, tiempo_actual
    carga_total = 10.0 if ddos_activado else slider_carga.val
    if ruido_activado:
        carga_total += np.random.normal(0, 0.05)
    carga_total = min(carga_total, pods_actuales * 2.0)
    uso_actual = carga_total / pods_actuales
    error = uso_actual - cpu_target
    kp = slider_kp.val
    if len(tiempos) - ultimo_escaleo >= cooldown_steps:
        delta = int(np.round(kp * error))
        deseadas = pods_actuales + delta
        deseadas = max(min_pods, min(max_pods, deseadas))
        if deseadas != pods_actuales:
            pods_actuales = deseadas
            ultimo_escaleo = len(tiempos)
    tiempos.append(tiempo_actual)
    uso_cpu.append(uso_actual * 100)
    cantidad_pods.append(pods_actuales)
    carga_registrada.append(carga_total)
    line_cpu.set_data(tiempos, uso_cpu)
    line_pods.set_data(tiempos, cantidad_pods)
    line_carga.set_data(tiempos, carga_registrada)
    text_cpu.set_text(f'CPU actual: {uso_actual * 100:.1f}%')
    text_cpu.set_position((tiempo_actual - 5, uso_actual * 100))
    text_pods.set_text(f'Pods activos: {pods_actuales}')
    text_pods.set_position((tiempo_actual - 5, pods_actuales))
    ax1.relim(); ax1.autoscale_view()
    ax2.relim(); ax2.autoscale_view()
    ax3.relim(); ax3.autoscale_view()
    tiempo_actual += dt

ani = FuncAnimation(fig, update, interval=200, cache_frame_data=False)